In [1]:
# This program performs Monte Carlo Simulation to calculate VaR.

import numpy as np
import pandas as pd
from pandas_datareader import data as web
import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import random
from scipy.stats import norm

random.seed(444)

assets = ['FB','AAPL','AMZN','GOOG','NFLX']

# import data
def get_data(stocks, start, end):
    stockData = yf.download(stocks,start,end)['Adj Close']
    df=pd.DataFrame(stockData)
    returns = df.pct_change().dropna()
    covMatrix = returns.cov()
    return returns,covMatrix

def port_mean(df,weights):
    return np.dot(df.mean(), weights)

def portfolioStd(df,weights):
    var=np.dot(weights.T,np.dot(df.cov(),weights))
    return np.sqrt(var)

endDate = datetime.datetime.now()
startDate = endDate - datetime.timedelta(days=500)

In [ ]:
df_per,covMatrix=get_data(assets, startDate, endDate)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-8:
Exception in thread Thread-6Traceback (most recent call last):
:
Traceback (most recent call last):
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
    self.run()
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 870, in run
  File "C:\Users\Acer\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Acer\anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
        return callee(*args, **kwargs)    
  File "C:\Users\Acer\anaconda3\lib\site-packages\yfinanc

In [ ]:
weights=np.array([0.1,0.1,0.1,0.1,0.1])
mean=port_mean(df_per,weights)
std=portfolioStd(df_per,weights)

In [ ]:
#Monte Carlo Simulation
mc_sims=100
T=50


meanM = np.full(shape=(T, len(weights)), fill_value=mean)
meanM = meanM.T

portfolio_sims = np.full(shape=(T, mc_sims), fill_value=0.0)

initialPortfolio = 10000

for m in range(mc_sims):
    Z = np.random.normal(size=(T, len(weights)))#uncorrelated RV's
    L = np.linalg.cholesky(covMatrix) #Cholesky decomposition to Lower Triangular Matrix
    dailyReturns = meanM + np.inner(L, Z) #Correlated daily returns for individual stocks
    portfolio_sims[:,m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)*initialPortfolio
    
plt.plot(portfolio_sims)
plt.ylabel('Portfolio Value ($)')
plt.xlabel('Days')
plt.title('MC simulation of a stock portfolio')
plt.show()


In [ ]:
def MonteVaR(series,alpha=5):
    if isinstance(series,pd.Series):
        return np.percentile(series,alpha)
    else:
        raise TypeError('Expected Pandas Data Series')
        
def MonteCVaR(series,alpha=5):
    if isinstance(series,pd.Series):
        belowVaR = series <=MonteVaR(series,alpha)
        below=series[belowVaR]
        return np.mean(below)
    else:
        raise TypeError('Expected Pandas Data Series')

In [ ]:
PortfolioReturn=pd.Series(portfolio_sims[-1,:])
alpha=5

In [ ]:

VaR = (initialPortfolio - MonteVaR(PortfolioReturn,alpha=5)).round(2)
CVaR =(initialPortfolio - MonteCVaR(PortfolioReturn,alpha=5))

print(f'VaR ${VaR}')
print(f'CVaR ${CVaR}')

In [ ]:
plt.plot(portfolio_sims,linewidth=1,alpha=0.5)
plt.ylabel('Portfolio Value ($)')
plt.xlabel('Days')
plt.axhline(MonteVaR(PortfolioReturn,alpha=5),linestyle='dashed',color='green',linewidth=1.5,label=f"{alpha}% VaR")
plt.axhline(MonteCVaR(PortfolioReturn,alpha=5),linestyle='dashed',color='red',linewidth=1.5,label=f"{alpha}% CVaR")
plt.legend()
plt.title('MC simulation of a stock portfolio')
plt.show()

In [ ]:
new_returns['Portfolio Return']=df_per.dot(weights)
